In [ ]:
import wandb

from model.make_model import *
import sys
sys.path.append('/home/thoresen/Documents/DeepLearning/02456-2021-project')
wandb.init(project='dtu-course', entity="freja-thoresen")
figure_path = '../figures/'

use_cuda=False

cuda=False
if torch.cuda.is_available():
  dev = "cuda:0"
  cuda=True
else:
  dev = "cpu"
device = torch.device(dev)

sequence_length = 10
epochs = 5 # 1000 epochs
learning_rate = 0.003
lstm_dropbout = 0.2
linear_dropout = 0.2
input_size = 1 # number of features
lstm_hidden_size = 20 # number of features in hidden state
linear_hidden_size = 40 # number of features in hidden state
num_layers = 2 # number of stacked lstm layers

num_classes = 1 # number of output classes

no_batches = 100
no_files = 100 # no. data files

wandb.config = {
    "learning_rate": learning_rate,
    "sequence_length": sequence_length,
    "epochs": epochs,
    "input_size": input_size,
    "lstm_hidden_size": lstm_hidden_size,
    "linear_hidden_size": linear_hidden_size,
    "num_layers": num_layers,
    "no_batches": no_batches,
    "no_files": no_files,
    "lstm_dropbout": lstm_dropbout,
    "linear_dropout": linear_dropout,
    "num_classes": num_classes,
    "input_size": input_size
}

sweep_config = {
    'name': 'data-sweep',
    'project': 'dtu-course',
    'method': 'grid',
    'metric': {
        'name': 'training_loss',
        'goal': 'minimize'
    }
}

parameters_dict = {
    'learning_rate': {
        'values': [0.0002, 0.0004, 0.0008]
    },
    'sequence_length': {
        'values': [10, 40, 80, 120, 200]
    },
    'epochs': {
        'value': 40
    },
    'input_size': {
        'value': input_size
    },
    'lstm_hidden_size': {
        'value': 20
    },
    'linear_hidden_size': {
        'value': 20
    },
    'num_layers': {
        'value': 2
    },
    'no_batches': {
        'values': [50, 100, 200, 400, 800]
    },
    'no_files': {
        'value': no_files
    },
    'lstm_dropbout': {
        'value': 0.2
    },
    'linear_dropout': {
        'value': 0.2
    },
    'num_classes': {
        'value': num_classes
    }
}

sweep_config['parameters'] = parameters_dict
sweep_id = wandb.sweep(sweep_config, project="dtu-course")
print(wandb.config)
def sweep():
    with wandb.init() as run:
        config = Config(wandb.config)
        lstm, criterion, optimizer = make_model(config)
        train_model(lstm, criterion, optimizer, config)
wandb.agent(sweep_id, function=sweep)